In [7]:
from pybaseball import statcast
import pandas as pd
from datetime import date, timedelta

season_start = "2025-03-20"
season_end = (date.today() + timedelta(days=1)).isoformat()

df = statcast(start_dt=season_start, end_dt=season_end)

This is a large query, it may take a moment to complete


100%|██████████| 70/70 [00:07<00:00,  8.79it/s]
/Users/jbrys/sportsbalf/.venv/lib/python3.9/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


In [8]:
df = df[df['pitch_type'].notnull()]
df['is_k'] = df['events'] == 'strikeout'
df['is_pa'] = df['description'].isin([
    'hit_into_play', 'hit_into_play_score', 'hit_into_play_no_out',
    'strikeout', 'walk', 'hit_by_pitch', 'swinging_strike',
    'foul', 'called_strike', 'bunt_foul', 'foul_tip'
])

k_by_park = df.groupby('home_team').agg({
    'is_k': 'sum',
    'batter': 'count'
}).reset_index()

k_by_park.columns = ['team_abbr', 'total_ks', 'total_events']
k_by_park['k_pct'] = k_by_park['total_ks'] / k_by_park['total_events']

league_avg_k = df['is_k'].sum() / df['batter'].count()

# Normalize to get K park factor
k_by_park['k_park_factor'] = k_by_park['k_pct'] / league_avg_k

k_by_park = k_by_park[['team_abbr', 'k_park_factor']]
k_by_park = k_by_park.sort_values('k_park_factor', ascending=False)

print("League Avg K%:", round(league_avg_k * 100, 2), "%")
display(k_by_park)

League Avg K%: 5.64 %


,team_abbr,k_park_factor
17,MIN,1.140288
13,LAA,1.108756
11,HOU,1.094613
10,DET,1.092226
23,SEA,1.064945
16,MIL,1.053896
22,SD,1.053407
1,ATL,1.051498
7,CLE,1.047694
18,NYM,1.046155


In [9]:
path = "../data/raw/fangraphs_park_factors_2025.csv"
k_by_park.to_csv(path, index=False)

In [1]:
from ingest.park_factors import compute_k_park_factors

compute_k_park_factors(start="2025-04-01")

This is a large query, it may take a moment to complete


/Users/jbrys/sportsbalf/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
100%|██████████| 56/56 [00:06<00:00,  9.15it/s]
